In [ ]:
#%%capture
%run '5_FeatureEngineering.ipynb'

In [52]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import imblearn
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing


# Logistic Regression (base?) - line model:
df = pd.read_csv('data/data.csv')
# create a new data frame without the NaN in our target feature
df_wo_target_nan = df.dropna(axis=0)

# Defining X and y
features = df_wo_target_nan.columns.tolist()
features.remove('bank_account')

X = df_wo_target_nan[features]
y = df_wo_target_nan.bank_account

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True)

#Oversampling
oversample = imblearn.over_sampling.RandomOverSampler(sampling_strategy='minority')
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

# Encode categorical features
enc = preprocessing.OrdinalEncoder()
X_train_fitted_transformed = enc.fit_transform(X_train_over)
X_test_fitted_transformed = enc.fit_transform(X_test)


### Using Logistic regression using non-scaled data

log_reg = LogisticRegression(max_iter=10000)
log_reg.fit(X_train_fitted_transformed, y_train_over)

y_pred_train = log_reg.predict(X_train_fitted_transformed)
y_pred = log_reg.predict(X_test_fitted_transformed)

# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train_over, y_pred_train), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
print("--------"*10)
# Print MAE of our model
y_pred_train_ = []
y_pred_ = []
for i in y_pred_train:
    if i == 'Yes': y_pred_train_.append(1)
    if i == 'No': y_pred_train_.append(0)
for i in y_pred:
    if i == 'Yes': y_pred_.append(1)
    if i == 'No': y_pred_.append(0) 
print("Mean absolute error on train set:", round(mean_absolute_error(y_train_over.replace (('Yes','No'), (1,0)), y_pred_train_),2))
print("Mean absolute error on test set:", round(mean_absolute_error(y_test.replace (('Yes','No'), (1,0)), y_pred_),2))
print("--------"*10)


Accuracy on train set: 0.74
Accuracy on test set: 0.72
--------------------------------------------------------------------------------
Mean absolute error on train set: 0.26
Mean absolute error on test set: 0.28
--------------------------------------------------------------------------------


In [50]:
# #y_train_over.replace (('Yes','No'), (1,0))
# y_pred_train_ = []
# for i in y_pred_train:
#     if i == 'Yes': y_pred_train_.append(1)
#     if i == 'No': y_pred_train_.append(0)

print (type(y_test))
print (type(y_train_over))


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# erst train - test - split, dann remove imbalance of data;

# Defining X and y
features = df_cleaned.columns.tolist()
features.remove('bank_account_Yes')

X = df_cleaned[features]
y = df_cleaned.bank_account_Yes

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True)

# Check the shape of the data sets
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

### Reduce imbalance of data

1. Random Oversampling: Randomly duplicate examples in the minority class.
2. Random Undersampling: Randomly delete examples in the majority class.

class imblearn.over_sampling.RandomOverSampler(*, sampling_strategy='auto', random_state=None, shrinkage=None)[source]

Class to perform random over-sampling.

### Preprocessing of data

In [ ]:
import imblearn
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score 

In [ ]:
#Oversampling
oversample = imblearn.over_sampling.RandomOverSampler(sampling_strategy='minority')
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

display('X_train: ' + str(X_train.shape))
display('y_train: ' + str(y_train.shape))
display('X_train_over: ' + str(X_train_over.shape))
display('y_train_over: ' + str(y_train_over.shape))


In [ ]:
count_yes, count_no = 0, 0

for i in y_train_over:
    if i == True: 
        count_yes += 1
    if i == False: 
        count_no += 1
print (count_yes, count_no)

In [ ]:
#Undersampling
undersample = imblearn.under_sampling.RandomUnderSampler(sampling_strategy='majority')
X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)

count_yes, count_no = 0, 0

for i in y_train_under:
    if i == True: 
        count_yes += 1
    if i == False: 
        count_no += 1
print (count_yes, count_no)

### Predictive Modelling

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_absolute_error

In [ ]:
X_train_over.head()

In [ ]:
### Using Logistic regression using non-scaled data

# Logistic Regression without the id field
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_over, y_train_over)

y_pred_train = log_reg.predict(X_train_over)
y_pred = log_reg.predict(X_test)

In [ ]:
# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train_over, y_pred_train), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
print("--------"*10)

Evaluation (see description)

The evaluation metric for this challenge is Mean Absolute error, where 1 indicates that the individual does have a bank account and 0 indicates that they do not. 

In [ ]:
import pandas as pd


In [ ]:
# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train_over, y_pred_train), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
print("--------"*10)

In [ ]:
# Print MAE of our model
print("Mean absolute error on train set:", round(mean_absolute_error(pd.DataFrame(map(int, y_train_over)), pd.DataFrame(map(int, y_pred_train))),2))
print("Mean absolute error on test set:", round(mean_absolute_error(pd.DataFrame(map(int, y_test)), pd.DataFrame(map(int, y_pred))),2))
print("--------"*10)

# Result: Model seems overfittet, da im Test-Data-Set mehr Fehler auftreten als im Trainingsdataset.
# Mean absolute error bedeutet: in wie vielen Fällen lag die Vorhersage Falsch?
# MAE = 1 - Accuracy

In [ ]:
print (pd.DataFrame(map(int, y_train_over)))

Show confusion matrix based on predicted results

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm_train = confusion_matrix(y_train_over, y_pred_train)
sns.heatmap(cm_train, cmap="YlGnBu", annot=True, fmt='d');


# Das ist für das Train - Datenset

In [ ]:
cm_test = confusion_matrix(y_test, y_pred)
sns.heatmap(cm_test, cmap="YlGnBu", annot=True, fmt='d');


### Improve Accuracy/MAE with Logistic Regression with Randomsearch CV

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import loguniform

In [ ]:
# Logistic Regression
log_reg_with_randomS= LogisticRegression(max_iter=10)

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)


# define search space
param_grid = { "solver" : ['newton-cg', 'lbfgs', 'liblinear'],      # Hier werden die Hyperparameter definiert, die untersucht
               "penalty" : ['none', 'l1', 'l2', 'elasticnet']}      # werden sollen


# define Random search
Random_search = RandomizedSearchCV(log_reg_with_randomS, param_grid, n_iter=500, scoring='accuracy', n_jobs=1, cv=cv, random_state=1)

# execute Random search
Random_search.fit(X_train_under, y_train_under)  

In [ ]:
y_pred_train_RS = Random_search.predict(X_train_under)
y_pred_RS = Random_search.predict(X_test)

print("Tuned hpyerparameters :(best parameters) ",Random_search.best_params_)
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred_RS), 2))

### Logistic regression with GridSearchCV

In [ ]:
import numpy as np

In [ ]:
C = np.logspace(0, 4, num=10)
penalty = ['l1', 'l2']
solver = ['liblinear', 'saga']

hyperparameters = dict(penalty=penalty, solver=solver)

gridsearch = GridSearchCV(log_reg, hyperparameters, scoring='accuracy',
                  cv=5, verbose=5, n_jobs=1)
gridsearch.fit(X_train_under,y_train_under)

In [ ]:
y_pred_train_GS = gridsearch.predict(X_train_under)
y_pred_GS = gridsearch.predict(X_test)

print("Tuned hpyerparameters (best parameters): ",gridsearch.best_params_)
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred_GS), 2))